In [ ]:
import julia
julia.install()

from pathlib import Path

# # Point to the top of the project relative to this script
def projectdir(*args):
    return str(Path.cwd().joinpath("..", "..", "..", *args).resolve())

def print_allocated_memory():
   print("{:.2f} GB".format(torch.cuda.memory_allocated() / 1024 ** 3))
   
from torch.utils.data import DataLoader
from julia import Main as jl
from sklearn.metrics import accuracy_score
import ipdb
import torch
# from torchvision.transforms import Lambda
from tqdm import tqdm

from torchvision.models import resnet50
from torchvision.models import ResNet50_Weights
from torchvision import models
from torchvision.models.feature_extraction import create_feature_extractor

class DDVFAStrategy():
    """DDVFA Strategy"""

    def __init__(self):
        jl.project_dir = projectdir()
        jl.eval("using Pkg; Pkg.activate(project_dir)")
        jl.eval("using AdaptiveResonance")
        jl.eval("art = DDVFA(rho_lb=0.4, rho_ub=0.75)")
        # jl.eval("art.config = DataConfig(0.0, 255.0, 28*28)")
        jl.eval("art.config = DataConfig(0, 1.0, 49)")
        
        rn = resnet50()
        self.mod = create_feature_extractor(rn, {'layer4': 'layer4'})
        self.mod = self.mod.to('cuda')
        self.mod.eval()
        self.weights = ResNet50_Weights.DEFAULT
        self.preprocess = self.weights.transforms()
        self.min = 10.0
        self.max = 70.0

    def ext_features(self, img):
        # print(rgbimg.shape)
        # prep = self.preprocess(rgbimg)
        img = img.to('cuda')
        prep = self.preprocess(img)
        features = self.mod(prep)['layer4']
        # avg_features = features.mean(dim=1).flatten(start_dim=1).detach().cpu().numpy()
        avg_features = features.detach().mean(dim=1).flatten(start_dim=1)
        avg_features = (avg_features - self.min) / (self.max - self.min) * 2 - 1
        avg_features = avg_features.sigmoid().cpu().numpy()
        # avg_features = features.mean(dim=1).flatten(start_dim=1).cpu().numpy()
        # ipdb.set_trace()
        # avg_features.flatten().detach().numpy()
        return avg_features

    def train(self, experience):
        train_dataset = experience.dataset
        t = experience.task_label
        train_data_loader = DataLoader(
            # train_dataset,
            dataset=train_dataset,
            pin_memory=True,
            # num_workers=1,
            # num_workers=4,
            # batch_size=64,
            batch_size=90,
            # transform=self.trans
        )
        print(experience.dataset.__len__())
        for mb in tqdm(train_data_loader):
            # ld = mb[0].to('cuda')
            data, labels, tasks = mb
            af = self.ext_features(data)
            jl.features = af
            jl.labels = labels.numpy()
            # ipdb.set_trace()
            jl.eval("train!(art, features, y=labels)")
            # for sample, label, task in mb:
            # for sample in mb:
            #     self.ext_features(sample)

        # for sample in train_dataset:
        # n_samples = len(train_dataset)
        # for ix in range(n_samples):
        # for ix in tqdm(range(n_samples)):
            # ls, ll, lt = train_dataset[ix]
            # self.ext_features(ls)
            # self.model(ls)

            # # ipdb.set_trace()
            # jl.sample = ls.flatten().numpy()
            # jl.label = ll
            # jl.t = lt
            # jl.eval("train!(art, sample, y=label)")

    def eval(self, experience):
        eval_dataset = experience.dataset
        t = experience.task_label
        # eval_data_loader = DataLoader(
        #     # eval_dataset, num_workers=4, batch_size=128
        #     eval_dataset, num_workers=4, batch_size=32
        # )
        # print(eval_data_loader)

        total = 0
        correct = 0
        # for mb in eval_data_loader:
        #     for sample, label in mb:
        #         jl.sample = sample.transpose()
        #         jl.label = label.numpy()
        #         y_hat = jl.eval("classify(art, sample)")

        eval_data_loader = DataLoader(
            # train_dataset,
            dataset=train_dataset,
            pin_memory=True,
            # num_workers=1,
            # num_workers=4,
            # batch_size=64,
            batch_size=90,
            # transform=self.trans
        )
        print(experience.dataset.__len__())
        for mb in tqdm(train_data_loader):
            # ld = mb[0].to('cuda')
            data, labels, tasks = mb
            af = self.ext_features(data)
            jl.features = af
            jl.labels = labels.numpy()
            # ipdb.set_trace()
            y_hats = jl.eval("train!(art, features, y=labels)")
            correct = torch.sum(y_hats == labels)

            # total += 1
            # if y_hat == label:
            #     correct += 1
        
        return correct/total
        # j.samples = mb
        # y_hats = j.eval("classify(art, samples)")
        # accuracy_score(y_test, y_hat)

print_allocated_memory()

In [1]:
import julia
julia.install()

In [2]:
from pathlib import Path

# # Point to the top of the project relative to this script
def projectdir(*args):
    return str(Path.cwd().joinpath("..", "..", "..", *args).resolve())

def print_allocated_memory():
   print("{:.2f} GB".format(torch.cuda.memory_allocated() / 1024 ** 3))
   
# from torch.utils.data import DataLoader
from julia import Main as jl
# from sklearn.metrics import accuracy_score
# import ipdb
import torch
# from torchvision.transforms import Lambda
# from tqdm import tqdm

from torchvision.models import resnet50, ResNet50_Weights
from torchvision import models
from torchvision.models.feature_extraction import create_feature_extractor


from avalanche.models import MultiTaskModule, BaseTemplate

class DDVFAModule(MultiTaskModule):
# class DDVFAModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        jl.project_dir = projectdir()
        jl.eval("using Pkg; Pkg.activate(project_dir)")
        jl.eval("using AdaptiveResonance")
        jl.eval("art = DDVFA(rho_lb=0.4, rho_ub=0.75)")
        # jl.eval("art.config = DataConfig(0.0, 255.0, 28*28)")
        jl.eval("art.config = DataConfig(0, 1.0, 49)")
        
        self.mod = resnet50().to('cuda:0')
        self.mod.eval()
        # self.mod = create_feature_extractor(rn, {'layer4': 'layer4'})
        self.mod = create_feature_extractor(self.mod, {'layer4': 'layer4'})
        self.mod = self.mod.to('cuda')
        # self.mod.cuda()
        self.def_weights = ResNet50_Weights.DEFAULT
        self.preprocess = self.def_weights.transforms()
        self.min = 10.0
        self.max = 70.0

        a = torch.zeros(3).to('cuda')
        b = torch.zeros(3).to('cuda')
        self.weights = [a, b]
        # self.weights = self.weights.to('cuda')

    def parameters(self):
        return self.weights

    def ext_features(self, img):
        with torch.no_grad():
            # img = img.to('cuda')
            prep = self.preprocess(img)
            features = self.mod(prep)['layer4']
            avg_features = features.detach().mean(dim=1).flatten(start_dim=1)
            avg_features = (avg_features - self.min) / (self.max - self.min) * 2 - 1
            avg_features = avg_features.sigmoid().cpu().numpy()
        return avg_features

    def adaptation(self, dataset):
        super().adaptation(dataset)
        # your adaptation goes here

    def forward_single_task(self, x, task_label):
        # your forward goes here.
        # task_label is a single integer
        # the mini-batch is split by task-id inside the forward method.
        # for mb in tqdm(train_data_loader):
        #     # ld = mb[0].to('cuda')
        # data
        af = self.ext_features(x)
        jl.features = af
        jl.labels = [task_label]*len(x)
        # ipdb.set_trace()
        y_hat = jl.eval("train!(art, features, y=labels)")
        # pass
        return y_hat

print_allocated_memory()

0.00 GB


In [3]:
from avalanche.models import MultiTaskModule

import ipdb
# import numpy as np
import torch
import numpy as np

class CustomMTModule(MultiTaskModule):
    def __init__(self, in_features, initial_out_features=2):
        super().__init__()
    
    def parameters(self):
        return torch.zeros(3)

    def adaptation(self, dataset):
        super().adaptation(dataset)
        # your adaptation goes here

    def forward_single_task(self, x, task_label):
        ipdb.set_trace()
        # your forward goes here.
        # task_label is a single integer
        # the mini-batch is split by task-id inside the forward method.
        pass
        # return np.array([task_label])

In [4]:
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive
from torchvision.transforms import Lambda

import torch

import debug

trans = Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x)

scenario = SplitMNIST(
    n_experiences=5,
    train_transform=trans,
    eval_transform=trans,
)

# MODEL CREATION
# model = SimpleMLP(num_classes=scenario.n_classes)
# model = CustomMTModule(28*28)
model = DDVFAModule()

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    # loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

# a = torch.zeros(3)
# b = torch.zeros(3)
# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
# cl_strategy = BaseTemplate(
    model,
    # SGD(model.parameters(), lr=0.001, momentum=0.9),
    # torch.optim.Adam([a, b], lr=0.001),
    torch.optim.Adam(model.parameters(), lr=0.001),
    CrossEntropyLoss(),
    # train_mb_size=500,
    train_mb_size=90,
    train_epochs=1,
    eval_mb_size=90,
    evaluator=eval_plugin
)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [6, 7]
-- >> Start of training phase << --
0it [00:00, ?it/s]

AssertionError: multi-head assumes mini-batches of 2 dimensions <batch, classes>

In [76]:
# model.mod.layer1._modules['0']._modules['conv1'].
model.parameters()[0].device

device(type='cuda', index=0)

In [86]:
%debug

> c:\users\sasha\anaconda3\envs\avalanche\lib\site-packages\torch\nn\modules\conv.py(459)_conv_forward()
    457                             weight, bias, self.stride,
    458                             _pair(0), self.dilation, self.groups)
--> 459         return F.conv2d(input, weight, bias, self.stride,
    460                         self.padding, self.dilation, self.groups)
    461 

> c:\users\sasha\anaconda3\envs\avalanche\lib\site-packages\torch\nn\modules\conv.py(463)forward()
    461 
    462     def forward(self, input: Tensor) -> Tensor:
--> 463         return self._conv_forward(input, self.weight, self.bias)
    464 
    465 class Conv3d(_ConvNd):

> c:\users\sasha\anaconda3\envs\avalanche\lib\site-packages\torch\nn\modules\module.py(1194)_call_impl()
   1192         if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks or _global_backward_hooks
   1193                 or _global_forward_hooks or _global_forward_pre_hooks):
-> 1194             retu